In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv
/kaggle/input/riiid-test-answer-prediction/example_test.csv
/kaggle/input/riiid-test-answer-prediction/questions.csv
/kaggle/input/riiid-test-answer-prediction/train.csv
/kaggle/input/riiid-test-answer-prediction/lectures.csv
/kaggle/input/riiid-test-answer-prediction/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/riiid-test-answer-prediction/riiideducation/__init__.py


In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

In [3]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

# Data

In [4]:
used_data_types_dict = {
    'row_id': 'int32',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float16',
    'prior_question_had_explanation': 'boolean'
}

train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                       usecols = used_data_types_dict.keys(),
                       dtype=used_data_types_dict,
                       index_col = 0,
                       nrows=10**6)

print(train_df.dtypes)
print(train_df.shape)
train_df.head()

timestamp                           int64
user_id                             int32
content_id                          int16
answered_correctly                   int8
prior_question_elapsed_time       float16
prior_question_had_explanation    boolean
dtype: object
(1000000, 6)


,timestamp,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,
0,0,115,5692,1,NaN,<NA>
1,56943,115,5716,1,36992.0,False
2,118363,115,128,1,55008.0,False
3,131167,115,7860,1,19008.0,False
4,137965,115,7922,1,11000.0,False


In [5]:
test_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_test.csv")

print(test_df.shape)
test_df.head()

(104, 11)


,row_id,group_num,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
0,0,0,0,275030867,5729,0,0,NaN,NaN,[],[]
1,1,0,13309898705,554169193,12010,0,4427,19000.0,True,NaN,NaN
2,2,0,4213672059,1720860329,457,0,240,17000.0,True,NaN,NaN
3,3,0,62798072960,288641214,13262,0,266,23000.0,True,NaN,NaN
4,4,0,10585422061,1728340777,6119,0,162,72400.0,True,NaN,NaN


In [6]:
submission_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv")

print(submission_df.shape)
submission_df.head()

(104, 3)


,row_id,answered_correctly,group_num
0,0,0.5,0
1,1,0.5,0
2,2,0.5,0
3,3,0.5,0
4,4,0.5,0


In [7]:
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')

print(questions.shape)
questions.head()

(13523, 5)


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [8]:
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')

print(lectures.shape)
lectures.head()

(418, 4)


,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


# Feature Engineering

### user_df

In [9]:
user_df = train_df[train_df.answered_correctly != -1].groupby('user_id') \
.agg({'answered_correctly': ['count', 'mean']}).reset_index()

user_df.columns = ['user_id', 'user_questions', 'user_mean']

print(user_df.shape)
user_df.head()

(3824, 3)


,user_id,user_questions,user_mean
0,115,46,0.695652
1,124,30,0.233333
2,2746,19,0.578947
3,5382,125,0.672000
4,8623,109,0.642202


In [10]:
user_lect = train_df.groupby(["user_id", "answered_correctly"]).size().unstack()

# columns:-1, 0, 1
user_lect.columns = ['Lecture', 'Wrong', 'Right']

print(user_lect.shape)
user_lect.head()

(3824, 3)


,Lecture,Wrong,Right
user_id,,,
115,NaN,14.0,32.0
124,NaN,23.0,7.0
2746,1.0,8.0,11.0
5382,3.0,41.0,84.0
8623,3.0,39.0,70.0


In [11]:
user_lect['Lecture'] = user_lect['Lecture'].fillna(0)
user_lect = user_lect.astype('Int64')
user_lect['watches_lecture'] = np.where(user_lect.Lecture > 0, 1, 0)
user_lect = user_lect.reset_index()
user_lect = user_lect[['user_id', 'watches_lecture']]

print(user_lect.shape)
user_lect.head()

(3824, 2)


,user_id,watches_lecture
0,115,0
1,124,0
2,2746,1
3,5382,1
4,8623,1


In [12]:
user_df = user_df.merge(user_lect, on = "user_id", how = "left")
del user_lect

print(user_df.shape)
user_df.head()

(3824, 4)


,user_id,user_questions,user_mean,watches_lecture
0,115,46,0.695652,0
1,124,30,0.233333,0
2,2746,19,0.578947,1
3,5382,125,0.672000,1
4,8623,109,0.642202,1


### content_df

In [13]:
content_df = train_df[train_df.answered_correctly != -1].groupby('content_id') \
.agg({'answered_correctly': ['count', 'mean']}).reset_index()

content_df.columns = ['content_id', 'content_questions', 'content_mean']

print(content_df.shape)
content_df.head()

(13076, 3)


,content_id,content_questions,content_mean
0,0,73,0.863014
1,1,55,0.927273
2,2,444,0.560811
3,3,199,0.798995
4,4,307,0.602606


### train_df

In [14]:
train_df = train_df.merge(user_df, on = "user_id", how = "left")
train_df = train_df.merge(content_df, on = "content_id", how = "left")

print(train_df.isnull().sum())
print(train_df.shape)
train_df.head()

timestamp                             0
user_id                               0
content_id                            0
answered_correctly                    0
prior_question_elapsed_time       23723
prior_question_had_explanation     3816
user_questions                        0
user_mean                             0
watches_lecture                       0
content_questions                 12319
content_mean                      12319
dtype: int64
(1000000, 11)


,timestamp,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,user_questions,user_mean,watches_lecture,content_questions,content_mean
0,0,115,5692,1,NaN,<NA>,46,0.695652,0,337.0,0.721068
1,56943,115,5716,1,36992.0,False,46,0.695652,0,242.0,0.768595
2,118363,115,128,1,55008.0,False,46,0.695652,0,181.0,0.961326
3,131167,115,7860,1,19008.0,False,46,0.695652,0,203.0,0.926108
4,137965,115,7922,1,11000.0,False,46,0.695652,0,171.0,0.959064


In [15]:
train_df = train_df[train_df.answered_correctly != -1]
mean_prior = train_df.prior_question_elapsed_time.astype("float64").mean()
label_enc = preprocessing.LabelEncoder()

train_df['content_questions'].fillna(0, inplace = True)
train_df['content_mean'].fillna(0.5, inplace = True)
train_df['user_questions'].fillna(0, inplace = True)
train_df['user_mean'].fillna(0.5, inplace = True)
train_df['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
train_df['prior_question_had_explanation'].fillna(False, inplace = True)
train_df['prior_question_had_explanation'] = label_enc.fit_transform(train_df['prior_question_had_explanation'])
train_df[['content_questions', 'user_questions']] = train_df[['content_questions', 'user_questions']].astype(int)

print(train_df.dtypes)
print(train_df.shape)
print('total null:', train_df.isnull().sum().sum())
train_df.head()

timestamp                           int64
user_id                             int32
content_id                          int16
answered_correctly                   int8
prior_question_elapsed_time       float16
prior_question_had_explanation      int64
user_questions                      int64
user_mean                         float64
watches_lecture                     int64
content_questions                   int64
content_mean                      float64
dtype: object
(980093, 11)
total null: 0


,timestamp,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,user_questions,user_mean,watches_lecture,content_questions,content_mean
0,0,115,5692,1,inf,0,46,0.695652,0,337,0.721068
1,56943,115,5716,1,36992.0,0,46,0.695652,0,242,0.768595
2,118363,115,128,1,55008.0,0,46,0.695652,0,181,0.961326
3,131167,115,7860,1,19008.0,0,46,0.695652,0,203,0.926108
4,137965,115,7922,1,11000.0,0,46,0.695652,0,171,0.959064


In [16]:
train_df, test_df = train_test_split(train_df, random_state=666, test_size=0.2)

print(train_df.shape)
print(test_df.shape)

(784074, 11)
(196019, 11)


In [17]:
features = [col for col in train_df.columns if col != 'answered_correctly']
target = 'answered_correctly'

print(features)
print(target)

['timestamp', 'user_id', 'content_id', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'user_questions', 'user_mean', 'watches_lecture', 'content_questions', 'content_mean']
answered_correctly


In [18]:
def create_model(trial):
    num_leaves = trial.suggest_int("num_leaves", 2, 31)
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 8)
    min_child_samples = trial.suggest_int('min_child_samples', 100, 1200)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0001, 0.99)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 5, 90)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.0001, 1.0)
    feature_fraction = trial.suggest_uniform('feature_fraction', 0.0001, 1.0)
    
    model = lgb.LGBMClassifier(
        num_leaves=num_leaves,
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_child_samples=min_child_samples, 
        min_data_in_leaf=min_data_in_leaf,
        learning_rate=learning_rate,
        feature_fraction=feature_fraction,
        random_state=666
    )
    return model

In [19]:
def objective(trial):
    model = create_model(trial)
    model.fit(train_df[features], train_df[target])
    score = roc_auc_score(
        test_df[target].values, 
        model.predict_proba(test_df[features])[:,1]
    )
    return score

In [20]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=70)
params = study.best_params

print(params)

[I 2020-12-09 12:53:25,468] A new study created in memory with name: no-name-2bb90b79-d80f-44e1-afe4-24efc772ce0c
[I 2020-12-09 12:53:32,587] Trial 0 finished with value: 0.7696863988599271 and parameters: {'num_leaves': 14, 'n_estimators': 245, 'max_depth': 6, 'min_child_samples': 985, 'learning_rate': 0.36574233989391514, 'min_data_in_leaf': 12, 'bagging_fraction': 0.25925829671865075, 'feature_fraction': 0.09066806100586555}. Best is trial 0 with value: 0.7696863988599271.
[I 2020-12-09 12:53:34,673] Trial 1 finished with value: 0.7656221225059533 and parameters: {'num_leaves': 8, 'n_estimators': 50, 'max_depth': 8, 'min_child_samples': 975, 'learning_rate': 0.3712704291326219, 'min_data_in_leaf': 34, 'bagging_fraction': 0.9345243397365558, 'feature_fraction': 0.00570701920229568}. Best is trial 0 with value: 0.7696863988599271.
[I 2020-12-09 12:53:40,151] Trial 2 finished with value: 0.7705530583273168 and parameters: {'num_leaves': 19, 'n_estimators': 121, 'max_depth': 7, 'min_chi

{'num_leaves': 24, 'n_estimators': 235, 'max_depth': 4, 'min_child_samples': 699, 'learning_rate': 0.36264935081127747, 'min_data_in_leaf': 76, 'bagging_fraction': 0.8361863818107145, 'feature_fraction': 0.8572234499089182}


In [21]:
'''
params = {
'num_leaves': 24,
'n_estimators': 299,
'max_depth': 5,
'min_child_samples': 189,
'learning_rate': 0.23952703210876622,
'min_data_in_leaf': 62,
'bagging_fraction': 0.3263951638326585,
'feature_fraction': 0.7424117808052549}

score: 0.7721987996629855
'''

"\nparams = {\n'num_leaves': 24,\n'n_estimators': 299,\n'max_depth': 5,\n'min_child_samples': 189,\n'learning_rate': 0.23952703210876622,\n'min_data_in_leaf': 62,\n'bagging_fraction': 0.3263951638326585,\n'feature_fraction': 0.7424117808052549}\n\nscore: 0.7721987996629855\n"

In [22]:
model = lgb.LGBMClassifier(**params)

model.fit(train_df[features], train_df[target])
print('LGB score: ', roc_auc_score(test_df[target].values, model.predict_proba(test_df[features])[:,1]))

LGB score:  0.7716237705925194


In [23]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.merge(user_df, on = "user_id", how = "left")
    test_df = test_df.merge(content_df, on = "content_id", how = "left")
    test_df['content_questions'].fillna(0, inplace = True)
    test_df['content_mean'].fillna(0.5, inplace = True)
    test_df['watches_lecture'].fillna(0, inplace = True)
    test_df['user_questions'].fillna(0, inplace = True)
    test_df['user_mean'].fillna(0.5, inplace = True)
    test_df['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
    test_df['prior_question_had_explanation'].fillna(False, inplace = True)
    test_df['prior_question_had_explanation'] = label_enc.fit_transform(test_df['prior_question_had_explanation'])
    test_df[['content_questions', 'user_questions']] = test_df[['content_questions', 'user_questions']].astype(int)
    test_df['answered_correctly'] =  model.predict(test_df[features])
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])